In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/edgaregurrola/Repos/observatorio-ciudades/accesibilidad-urbana/aup/data.py:24: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


In [2]:
# Compu Edgar
grl_dir = "../../../data/processed/santiago/"

In [3]:
alameda = gpd.read_file(grl_dir + "buffer 800m nueva alameda.shp")

In [4]:
alameda = alameda.to_crs("EPSG:4326")
hex_gdf = aup.create_hexgrid(alameda, 10)
print(hex_gdf.shape)
hex_gdf.head(2)

(1096, 2)


,hex_id_10,geometry
0,8ab2c556a677fff,"POLYGON ((-70.63450 -33.43576, -70.63513 -33.4..."
1,8ab2c556a0c7fff,"POLYGON ((-70.63315 -33.43086, -70.63378 -33.4..."


In [5]:
'''schema = 'projects_research'
table = 'santiago_nodesproximity_4_5_kmh'
source = 'edu_basica_pub'
poly_wkt = alameda.dissolve().geometry.to_wkt()[0]
query = f"SELECT * FROM {schema}.{table} WHERE ST_Intersects(geometry, 'SRID=4326;{poly_wkt}') AND \"source\" = \'{source}\'"
gdf_download = aup.gdf_from_query(query)'''

'schema = \'projects_research\'\ntable = \'santiago_nodesproximity_4_5_kmh\'\nsource = \'edu_basica_pub\'\npoly_wkt = alameda.dissolve().geometry.to_wkt()[0]\nquery = f"SELECT * FROM {schema}.{table} WHERE ST_Intersects(geometry, \'SRID=4326;{poly_wkt}\') AND "source" = \'{source}\'"\ngdf_download = aup.gdf_from_query(query)'

In [6]:
nodes_prox = gpd.read_file(grl_dir + 'santiago_nodesproximity_format_4_5_kmh.gpkg')
nodes_prox.head(2) 

,osmid,supermercado_time,supermercado_count_15min,clinica_priv_time,clinica_priv_count_15min,clinica_pub_time,clinica_pub_count_15min,hospital_priv_time,hospital_priv_count_15min,hospital_pub_time,...,librerias_time,librerias_count_15min,local_mini_market_time,local_mini_market_count_15min,bakeries_time,bakeries_count_15min,restaurantes_bar_cafe_time,restaurantes_bar_cafe_count_15min,city,geometry
0,386138,5.337897,8,0.416410,19,24.706454,0,6.773174,1,12.855830,...,2.862839,21,2.206940,79,0.415357,38,0.415357,153,Santiago,POINT (-70.64625 -33.44290)
1,386139,4.304996,10,0.525741,21,23.598733,0,8.051900,1,14.134556,...,2.665707,22,2.221295,83,1.300392,42,1.694083,155,Santiago,POINT (-70.64725 -33.44313)


In [7]:
nodes_prox = nodes_prox[['osmid','supermercado_time','supermercado_count_15min','geometry']].copy()

In [65]:
hex_prox = aup.group_by_hex_mean(nodes_prox, hex_gdf, 10, 
                                 ['supermercado_time','supermercado_count_15min'],'hex_id_10', osmid=True)
hex_prox.loc[hex_prox.supermercado_time==0,'supermercado_count_15min'] = np.nan
hex_prox.loc[hex_prox.supermercado_time==0,'supermercado_time'] = np.nan

In [66]:
import h3
def neighbour_mean(hex_id, hex_id_name, hex_bins, col_name):
    return hex_bins.loc[hex_bins[hex_id_name].isin(h3.k_ring(hex_id,1)),col_name].mean()

In [67]:
hex_idxs = h3.k_ring(hex_prox.loc[pd.isna(hex_prox.supermercado_time),'hex_id_10'].sample(1).iloc[0],1)
# hex_prox.loc[hex_prox.hex_id_10.isin(hex_idxs)].supermercado_count_15min.mean()

In [73]:
hex_prox.loc[hex_prox.hex_id_10.isin(hex_idxs)]

,hex_id_10,geometry,supermercado_time,supermercado_count_15min
188,8ab2c55512c7fff,"POLYGON ((-70.71644 -33.45783, -70.71708 -33.4...",6.016352,1.0
448,8ab2c55512cffff,"POLYGON ((-70.71563 -33.45863, -70.71626 -33.4...",NaN,1.0
1039,8ab2c55512dffff,"POLYGON ((-70.71699 -33.45886, -70.71762 -33.4...",7.897747,1.0
716,8ab2c55512effff,"POLYGON ((-70.71509 -33.45761, -70.71572 -33.4...",4.819466,1.0
60,8ab2c555c597fff,"POLYGON ((-70.71482 -33.45943, -70.71545 -33.4...",5.955766,1.0
699,8ab2c555c5b7fff,"POLYGON ((-70.71428 -33.45841, -70.71491 -33.4...",6.295298,1.0
127,8ab2c555e967fff,"POLYGON ((-70.71617 -33.45966, -70.71681 -33.4...",5.863535,1.0


In [69]:
hex_prox.loc[hex_prox.hex_id_10.isin(hex_idxs)].supermercado_count_15min.mean()

1.0

In [70]:
hex_prox.loc[pd.isna(hex_prox.supermercado_count_15min)]

,hex_id_10,geometry,supermercado_time,supermercado_count_15min
363,8ab2c554121ffff,"POLYGON ((-70.65045 -33.44496, -70.65108 -33.4...",NaN,NaN
242,8ab2c55412d7fff,"POLYGON ((-70.65153 -33.44700, -70.65216 -33.4...",NaN,NaN
992,8ab2c554142ffff,"POLYGON ((-70.66370 -33.43970, -70.66434 -33.4...",NaN,NaN
173,8ab2c5541477fff,"POLYGON ((-70.66289 -33.44050, -70.66352 -33.4...",NaN,NaN
326,8ab2c5541497fff,"POLYGON ((-70.66965 -33.44163, -70.67028 -33.4...",NaN,NaN
...,...,...,...,...
1070,8ab2c555e867fff,"POLYGON ((-70.71644 -33.46250, -70.71708 -33.4...",NaN,NaN
746,8ab2c555e937fff,"POLYGON ((-70.72050 -33.45851, -70.72113 -33.4...",NaN,NaN
819,8ab2c555e94ffff,"POLYGON ((-70.71590 -33.46148, -70.71653 -33.4...",NaN,NaN
1090,8ab2c556a2effff,"POLYGON ((-70.62612 -33.43622, -70.62675 -33.4...",NaN,NaN


In [71]:
hex_prox.loc[pd.isna(hex_prox.supermercado_count_15min),'supermercado_count_15min'] = hex_prox.loc[pd.isna(hex_prox.supermercado_count_15min)].apply(lambda x: neighbour_mean(x['hex_id_10'],
                                                                                                                                                   'hex_id_10',
                                                                                                                                                   hex_prox,
                                                                                                                                                   'supermercado_count_15min'), axis=1)

In [72]:
hex_prox.loc[pd.isna(hex_prox.supermercado_count_15min),'supermercado_count_15min']

Series([], Name: supermercado_count_15min, dtype: float64)